In [1]:
import random

import numpy as np
import matplotlib.pyplot as plt

# Bokeh imports
from bokeh.io import output_notebook, show, save
from bokeh.plotting import figure, output_file, ColumnDataSource, reset_output
from bokeh.models import HoverTool
output_notebook()

from ipywidgets.widgets import Text

#scipy constants
from scipy.constants import Boltzmann as kB
from scipy.constants import Avogadro as NA
from scipy.constants import Planck as h
from scipy.constants import speed_of_light as c0
from scipy.constants import R

Loading BokehJS ...

### Set Global parameters

In [2]:
L_nm = 300  # length of the nanotube
R_nm = 1  # radius of the exiton
N_DEF = 10  # number of defects per nanotube
T_STEP_ps = 1 # time step
D_exc_nm_per_s = 1.07e15  # Exciton Diffusion Coefficient https://doi.org/10.1021/nn101612b

k_r_per_s = 1e10  # radiativ decay constant
k_nr_per_s = 1e11  # non-radiativ decay constant bright
k_d_per_s = 1e11  # constant for transion into dark state
k_n_per_s = 1e11


## Exciton Fate

Very small MC simulation without any dynamic elements. The exciton is allowed to radiative, non-ratidative decays as well as diffusion and falling into dark state (not in agreement with working modells) More for algorithm purposes

In [3]:
def exciton_fate():
    """Determines the fate of a single exciton.
    
    Returns
    -------
    exciton_fate : 1D array
        Matrix showing the binned fate of the MC steps. Order is
        k_r_per_s, k_nr_per_s, k_d_per_s, k_d_per_s
    """
    kin_const = [k_r_per_s, k_nr_per_s, k_d_per_s, k_d_per_s]
    fate = 3  # fate of single exciton in MC step
    exciton_fate = np.zeros(len(kin_const))
    
    while fate > 1:
        # calculate the probability for exciton decay
        p_fate = np.array([e * random.uniform(0, 1) for e in kin_const])
        
        # Store result for highest probability
        fate = p_fate.argmax()
        exciton_fate[fate] +=1
        
        # insurance that there won't be an endless loop
        if exciton_fate.sum() > 1e3:
            print('Simulation exceeds 1e3 steps, loop aborded')
            return exciton_fate
    return exciton_fate


In [4]:
def photons_fate(n_photons, func, func_kwargs={}):
    """Simulate the fate of numerous photons.
    For a key to the fate look a the respective exciton fate function.
    
    Parameters
    ----------
    n_photons : int
        Number of photons to be used in the simulation.
    func : callable
        Function to perform MC simulation of a single exciton.
    func_kwargs: dict
        Dictionary containing the arguments and keyword arguments for
        func.

    Returns
    -------
    photons_fate : 1D array
        Binnend fate of the photons in simulation.
    quantum_yield : 1D array
        Quantum yield of the non-radiative and radiative decay."""
    info = Text()
    display(info)
    
    # initiate matrix size
    photons_fate = func(**func_kwargs)
    info.value = f"Processing photon (1/ {n_photons})"
    
    # loop for the desired number of photons
    for p in np.arange(n_photons-1):
        photons_fate += func(**func_kwargs)
        info.value = f"Processing photon ({p+2}/ {n_photons})"
    
    # calculate the quantum yield
    quantum_yield = photons_fate[:2] / n_photons
    
    return photons_fate, quantum_yield

In [5]:
photons_fate(1e3, exciton_fate)

Text(value='')

(array([  1., 999., 930., 959.]), array([0.001, 0.999]))

## Modell change

E11 can decay radiativ and non radiative, as well as transition to E11* (defect states) which can also decay radiative and non radiative. If the exciton gets trapped within E11* is determined with a 1D random walk

E11* transitions are marked with d for 'defect'

### Set Global Parameters

In [6]:
TAU_ps = 100
TAU_d_ps = 1000

k_r_per_s = 1e11  # constant for radiativ decay from E11
k_d_r_per_s = 1e10  # constant for radiativ decay from E11*
k_nr_per_s = 1e12  # constant of non-radiativ decay from E11
k_d_nr_per_s = 1e12  # constant for non-radiativ decay from E11*
k_nothing = lambda t_step: (k_r_per_s + k_nr_per_s) * TAU_ps / t_step;
k_nothing_d = lambda t_step: (k_d_r_per_s + k_d_nr_per_s) * TAU_d_ps / t_step;

In [7]:
def create_defects(l=L_nm, n_def=N_DEF):
    """Creates defects along the CNT at random position.
    
    Parameters
    ----------
    l : int, optional
        Length of the CNT in nm, global constant as default.
    n_def : int, optional
        Number of defects on the CNT, global constant as default.
    
    Returns
    -------
    pos_def : 1D array
        Positions in nm of the defects on the CNT stored in
        array size (n_def, 1)    
    """
    return np.random.randint(0, l, size=(n_def, 1))

In [8]:
def create_exciton(l=L_nm):
    """Creates exciton on the CNT at random position.
    
    Parameters
    ----------
    l : int, optional
        Length of the CNT in nm, global constant as default.
    
    Returns
    -------
    pos_exc : int
        Position of the exciton along the CNT as a random integer."""
    return random.randrange(l)

In [9]:
def exciton_walk(t_step, n_defects=10, l=L_nm):
    """
    
    Parameters
    ----------
    t_step : float
        Timestep in ps.
    n_defects : int, optional
        Number of defects on CNT. Default is 10
    l : int, optional
        Length of the CNT in nm, global constant as default.
        
    Returns
    -------
    exciton_fate : 1D array
        Array contains the binned fate of the exciton for each MC step:
        fate = 0 : E11 radiative decay
        fate = 1 : E11* radiative decay
        fate = 2 : E11 non-radiative decay
        fate = 3 : E11* non-radiative decay
        fate = 4 : Free diffusion walk
        fate = 5 : Exciton stays in trap
        fate = 6 : Exciton becomes trapped        
    """
    
    kin_const = [k_r_per_s, k_d_r_per_s, k_nr_per_s, k_d_nr_per_s,
                k_nothing(t_step), k_nothing_d(t_step)]
    
    # inital exciton is free in E11
    fate = 4
    trapped = 0
    
    # Initiate matrix to store exciton fate
    exciton_fate = np.zeros(len(kin_const)+1)
    
    # Inital position of the exciton and defects
    pos_exc_0 = create_exciton(l)
    defects = create_defects(l, n_defects)
    
    while fate > 3:
        
        # step if exciton is free
        if trapped == 0:
            pos_exc_1 = round(pos_exc_0 + (
                2 * D_exc_nm_per_s * t_step * 1e-12)**0.5)
        
        # quenching of the exciton at tube end
        if pos_exc_1 >= l:
            fate = 2
            exciton_fate[fate] += 1
            break
        
        # check if exciton became trapped
        if trapped == 0:
            pathway = np.arange(pos_exc_1-pos_exc_0)
            if np.in1d(pathway, defects).any():
                trapped = 1
                fate = 6
                exciton_fate[fate] += 1
        
        # fate of a trapped exciton
        if trapped == 1:
            # calculate probability for fate of trapped exciton
            p_fate = np.array([e * random.uniform(0, 1)
                               for e in kin_const[1::2]])
            # Store result for highest probability
            fate = (2*p_fate.argmax() + 1)
            exciton_fate[fate] +=1
        
        # fate of freely diffusing exciton
        else:
            # calculate probability for fate of free exciton
            p_fate = np.array([e * random.uniform(0, 1)
                               for e in kin_const[::2]])
            # Store result for highest probability
            fate = p_fate.argmax() * 2
            exciton_fate[fate] +=1
        
        # insurance that there won't be an endless loop
        if exciton_fate.sum() > 1e6:
            print('Simulation exceeds 1e6 steps, loop aborded')
            return exciton_fate
        
        # set position to new starting position
        pos_exc_0 = pos_exc_1
    
    return exciton_fate

In [10]:
photons_fate(1000, exciton_walk, {'t_step':1})

Text(value='')

(array([0.000000e+00, 0.000000e+00, 3.330000e+02, 6.670000e+02,
        5.450000e+02, 1.446571e+06, 6.670000e+02]), array([0., 0.]))